In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('myData/'+'231008_0119_2kPa_2line_Data'+'.csv')
data=data.iloc[:,:28]
data.describe()

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx26,Sx27,Sx30,Sx31,Sx32,Sx33,Force,z,theta,r
count,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,...,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000,115654.000000
mean,47.789284,46.587351,55.058288,43.507158,51.653830,64.754176,60.831080,53.016065,62.286681,56.452334,...,47.492486,54.471498,50.516973,55.370615,55.781778,49.804991,39.021419,80.001453,170.042368,3.200062
std,1.435331,1.708339,1.499085,1.786158,1.199627,1.697183,1.966231,1.891453,1.799997,1.444436,...,2.180379,3.001172,2.486547,1.726292,1.757992,1.515384,30.667866,48.438790,103.739277,1.600254
min,41.300000,40.700000,47.700000,32.800000,43.300000,45.700000,46.800000,44.600000,50.700000,47.200000,...,37.500000,45.900000,42.700000,48.200000,46.400000,39.000000,-455.760000,0.000000,0.000000,0.800000
25%,46.900000,45.500000,54.100000,42.300000,50.900000,64.000000,60.000000,51.800000,61.200000,55.500000,...,46.300000,52.800000,49.100000,54.200000,54.700000,48.800000,11.070000,40.000000,80.000000,1.600000
50%,47.800000,46.600000,55.100000,43.500000,51.700000,64.900000,61.000000,52.900000,62.200000,56.500000,...,47.300000,54.000000,50.200000,55.300000,55.800000,49.800000,33.260000,80.000000,180.000000,3.200000
75%,48.700000,47.600000,56.100000,44.600000,52.400000,65.700000,62.000000,54.000000,63.200000,57.400000,...,48.300000,55.300000,51.400000,56.400000,56.900000,50.800000,59.360000,120.000000,260.000000,4.800000
max,54.500000,55.400000,61.900000,52.200000,58.000000,69.300000,66.400000,63.900000,71.100000,66.800000,...,63.400000,70.900000,66.700000,66.700000,65.700000,56.800000,134.500000,160.000000,340.000000,5.600000


In [16]:
# SDreal 값이 999이거나 100이상인 이상치 제거
data = data[~(data == 999).any(axis=1)]
data = data[~(data == 255).any(axis=1)]
data = data[~(data == 4).any(axis=1)]

# SDreal에서 결측값 제거
data = data[~data.isna().any(axis=1)]

# 'Force', 'theta', 'z', 'r' 열을 제외한 열에서 100 이상의 값을 제거
cols_to_exclude = ["Force", "theta", "z", "r"]
cols_to_check = [col for col in data.columns if col not in cols_to_exclude]

data = data[~((data < 0) & (data.columns == "Force")).any(axis=1)]
data = data[~((data >= 10000) & (data.columns == "Force")).any(axis=1)]
data.describe()

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx10,Sx11,...,Sx26,Sx27,Sx30,Sx31,Sx32,Sx33,Force,z,theta,r
count,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,...,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000,99122.000000
mean,47.785374,46.565754,55.053824,43.490999,51.653430,64.754710,60.839275,53.022690,62.273591,56.440344,...,47.479504,54.451611,50.504461,55.366824,55.774105,49.784367,37.789275,79.997902,170.048223,3.066750
std,1.436813,1.708538,1.496517,1.783120,1.205012,1.686369,1.947581,1.893671,1.793562,1.451408,...,2.149145,2.983091,2.460390,1.731227,1.757833,1.522854,30.908831,48.439815,103.747361,1.692098
min,41.300000,40.700000,47.700000,32.800000,43.300000,45.700000,46.800000,44.600000,50.700000,47.200000,...,37.500000,45.900000,42.700000,48.200000,46.400000,39.000000,4.010000,0.000000,0.000000,0.800000
25%,46.800000,45.500000,54.100000,42.300000,50.900000,64.000000,60.000000,51.800000,61.200000,55.500000,...,46.300000,52.800000,49.100000,54.200000,54.600000,48.800000,8.650000,40.000000,80.000000,1.600000
50%,47.800000,46.500000,55.100000,43.500000,51.700000,64.900000,61.000000,52.900000,62.100000,56.500000,...,47.300000,54.000000,50.200000,55.300000,55.800000,49.800000,29.830000,80.000000,180.000000,2.400000
75%,48.700000,47.600000,56.000000,44.600000,52.400000,65.700000,62.000000,54.000000,63.200000,57.400000,...,48.300000,55.300000,51.400000,56.400000,56.900000,50.800000,58.790000,120.000000,260.000000,4.800000
max,54.500000,55.400000,61.900000,52.200000,58.000000,69.300000,66.400000,63.900000,71.100000,66.800000,...,63.400000,70.900000,64.900000,66.700000,65.700000,56.800000,134.500000,160.000000,340.000000,5.600000


In [17]:

# theta를 cos(theta)와 sin(theta)로 변환
data['cos_theta'] = np.cos(np.radians(data['theta']))
data['sin_theta'] = np.sin(np.radians(data['theta']))

# 특징(X)과 타겟(y) 분리
X = data.drop(['Force', 'z', 'theta', 'r', 'cos_theta', 'sin_theta'], axis=1)
y = data[['Force', 'z', 'cos_theta', 'sin_theta']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 특징 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 구성
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=X_train_scaled.shape[1:]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)  # 출력층은 타겟 변수의 개수와 동일
])

# 모델 컴파일
model.compile(loss='mse', optimizer='adam')

# 모델 훈련
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# 모델 평가
mse = model.evaluate(X_test_scaled, y_test)
rmse = np.sqrt(mse)
print(f"Test RMSE: {rmse}")

Epoch 1/100
2231/2231 [==============================] - 2s 856us/step - loss: 497.6189 - val_loss: 351.5476
Epoch 2/100
2231/2231 [==============================] - 2s 775us/step - loss: 317.6920 - val_loss: 300.7585
Epoch 3/100
2231/2231 [==============================] - 2s 782us/step - loss: 284.7298 - val_loss: 281.1657
Epoch 4/100
2231/2231 [==============================] - 2s 774us/step - loss: 269.5657 - val_loss: 271.1271
Epoch 5/100
2231/2231 [==============================] - 2s 766us/step - loss: 258.8900 - val_loss: 262.8168
Epoch 6/100
2231/2231 [==============================] - 2s 789us/step - loss: 250.6432 - val_loss: 253.4369
Epoch 7/100
2231/2231 [==============================] - 2s 788us/step - loss: 243.6046 - val_loss: 250.9827
Epoch 8/100
2231/2231 [==============================] - 2s 837us/step - loss: 237.7339 - val_loss: 244.0184
Epoch 9/100
2231/2231 [==============================] - 2s 799us/step - loss: 233.0082 - val_loss: 237.9509
Epoch 10/100
2231/2

2231/2231 [==============================] - 2s 815us/step - loss: 166.6194 - val_loss: 183.5278
Epoch 76/100
2231/2231 [==============================] - 2s 878us/step - loss: 166.2955 - val_loss: 183.2197
Epoch 77/100
2231/2231 [==============================] - 2s 795us/step - loss: 165.8844 - val_loss: 182.0486
Epoch 78/100
2231/2231 [==============================] - 2s 830us/step - loss: 165.6126 - val_loss: 182.3911
Epoch 79/100
2231/2231 [==============================] - 2s 841us/step - loss: 165.4896 - val_loss: 182.2980
Epoch 80/100
2231/2231 [==============================] - 2s 806us/step - loss: 165.1606 - val_loss: 182.6847
Epoch 81/100
2231/2231 [==============================] - 2s 814us/step - loss: 165.1546 - val_loss: 185.8140
Epoch 82/100
2231/2231 [==============================] - 2s 804us/step - loss: 164.5825 - val_loss: 180.2442
Epoch 83/100
2231/2231 [==============================] - 2s 805us/step - loss: 164.4732 - val_loss: 183.0936
Epoch 84/100
2231/2231 